<a href="https://colab.research.google.com/github/Gruppe3VDL/Gruppe3VDLws2019/blob/master/exercise2/task2/Task_2_DogBreedsIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__Complete all sub-tasks marked with ## TO DO! ## and submit the filled notebook on OLAT__ \
__Using a GPU is recommended here__

### Transfer Learning ###
Aim of this notebook is to implement the concept of transfer learning to train a bigger dataset. We try to compete on a well-known competiton on Kaggle known as Dog Breeds Identification. Read more about it here:

https://www.kaggle.com/c/dog-breed-identification/overview



To train a model on the Dog breeds dataset using transfer learning and submit your results to Kaggle.
Note: Below notebook gives some tips to run the code in pytorch. 

In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [0]:
###########################################################################################
## Register on Kaggle with your respepective group name                                  ##
##                                                                                       ##
##                                                                                       ##
##         Group Name:  WS19_VDL_GROUP_03                                                ##
##        Kaggle Page:  https://www.kaggle.com/ws19vdlgroup03                            ##
##                                                                                       ##
##                                                                                       ##
###########################################################################################
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import shutil

use_cuda = torch.cuda.is_available()

In [0]:
###########################################################################################
## Download the Dog-Breeds dataset in folder "data" from the Kaggle competition link     ##
## mentioned above.                                                                      ##
###########################################################################################
#!mkdir /root/.kaggle
#!echo '{"username":"ws19vdlgroup03","key":"820848a6160b4d9eeb38e21cb8ba74c3"}' > /root/.kaggle/kaggle.json
#!pip install kaggle
#!kaggle competitions download -c dog-breed-identification
#!unzip 'train.zip'
#!unzip 'test.zip'
#!unzip 'labels.csv.zip'

In [0]:
###########################################################################################
## We saved the downloaded dataset in Google Drive. Here, we define the paths to data    ##
## which can then be used in this notebook on Google Colab                               ##
###########################################################################################
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Navigate to project location
%cd '/content/drive/My Drive/TUK/Very Deep Learning/Exercises/exercise2/task2/'

# Define dataset locations
dir_root = "data/"
dir_train = dir_root + "train/"
dir_test = dir_root + "test/"

In [0]:
# import os

# def getListOfFiles(dirName):
#   # create a list of file and sub directories 
#   # names in the given directory 
#   listOfFile = os.listdir(dirName)
#   allFiles = list()

#   # Iterate over all the entries
#   for entry in listOfFile:
#     # Create full path
#     fullPath = os.path.join(dirName, entry)

#     # If entry is a directory then get the list of files in this directory 
#     if os.path.isdir(fullPath):
#       allFiles = allFiles + getListOfFiles(fullPath)
#     else:
#       allFiles.append(fullPath)
              
#   return allFiles

# seen = []
# duplicates = []

# im = getListOfFiles("data/test/")
# for i in im:
#   if "(1)" in i:
#     duplicates += [i]
#   else:
#     seen += [i]

# print(len(seen))
# print(len(duplicates))

# # Delete all duplicates
# for i in duplicates:
#   os.remove(i)

In [0]:
###########################################################################################
## Make your dataset to and dataloaders for the  test data                               ##
###########################################################################################
import torchvision
import torchvision.transforms as transforms


# Define data loader
def load_dataset(data_path):
    dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                      torchvision.transforms.ToTensor()]))
    dataset = torch.utils.data.DataLoader(
        dataset,
        batch_size=256,
        num_workers=0,
        shuffle=True
    )
    return dataset

# Load test data
test_dataset = load_dataset(dir_test)

In [0]:
###########################################################################################
## Split train data into 20% validation set and make dataloaders for train and val split ##
###########################################################################################
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Load train data and labels
train_dataset = load_dataset(dir_train)
train_labels = pd.read_csv(dir_root + 'labels.csv')

# Split labels using 80/20 ratio
y_train, y_validation = train_test_split(train_labels, test_size=0.20, random_state=42)

# Print shapes for verification
print("Train Set", y_train.shape)
print("Validation Set", y_validation.shape)

In [0]:
###########################################################################################
## HINT: One can (1) make their own custom dataset and dataloaders using the CSV file or ##
## (2) convert the Dog-breed training dataset into Imagenet Format, where all images of  ##
## one class are in a folder named with class as in the below given format. Standard     ##
## Pytorch Datasets and Dataloaders can then be applied over them                        ##
##   Root                                                                                ##
##   |                                                                                   ##
##   |---Class1 ___Img1.png                                                              ##
##   |          ___Img2.png                                                              ##
##   |                                                                                   ##
##   |---Class2 ___Img3.png                                                              ##
##   |          ___Img4.png                                                              ##
##   |....                                                                               ##
##   |....                                                                               ##
###########################################################################################
import os
from shutil import copyfile


# We are going for option (2) given in hint above, by manually converting whole training
# dataset to Imagenet Format. Two separate datasets in Imagenet Format from the training
# and validation split above are created. Not very optimal, takes ages to run.
dir_converted_train = dir_root + "converted/train/"
dir_converted_valid = dir_root + "converted/valid/"

ids = train_labels['id'].to_numpy()
breeds = train_labels['breed'].to_numpy()

if not os.path.exists(dir_converted_train):
  os.makedirs(dir_converted_train)

if not os.path.exists(dir_converted_valid):
  os.makedirs(dir_converted_valid)

print("Total Images:", len(breeds))
for i in range(0, len(breeds)):
  path = dir_converted_train + breeds[i] + "/"
  if ids[i] in y_validation.to_numpy()[:, 0]:
    path = dir_converted_valid + breeds[i] + "/"
  
  if not os.path.exists(path):
    os.makedirs(path)

  src = dir_train + "train_images/" + ids[i] + ".jpg"
  out = path + ids[i] + ".jpg"
  # copyfile(src, out) # UNCOMMENT THIS LINE TO CONVERT DATASET
  print("\rConverted: {}%".format(round(i/len(breeds) * 100, 2)), end='')

__Train famous Alexnet model on Dog breeds dataset. It is not easy to train the alexnet model from 
scratch on the Dog breeds data itself. Curious minds can try for once to train Alexnet from scratch. We adopt Transfer Learning here. We 
obtain a pretrained Alexnet model trained on Imagenet and apply transfer learning to it to get better results.__

## Transfer Learning

In [0]:
###########################################################################################
## Freeze the weigths of the pretrained alexnet model and change the last classification ##
## layer from 1000 classes of Imagenet to 120 classes of Dog Breeds, only classification ##
## layer should be unfreezed and trainable                                               ##
###########################################################################################
import torchvision.models as models

from train_test import start_train_test
from collections import OrderedDict


class AlexNet():
  def __init__(self, num_classes=120, inputs=3):
    # Use PyTorch's pretrained AlexNet
    self.pretrained_model = models.alexnet(pretrained=True)

    # Freeze weights on all layers
    for name, param in self.pretrained_model.named_parameters():
        param.requires_grad = False

    # Replace last layer (new layer has requires_grad=True by default)
    in_params = self.pretrained_model.classifier[6].in_features
    self.pretrained_model.classifier[6] = nn.Linear(in_params, num_classes)
    self.pretrained_model.cuda()

class ResNet18():
  def __init__(self, num_classes=120, inputs=3):
    # Use PyTorch's pretrained AlexNet
    self.pretrained_model = models.resnet18(pretrained=True)

    # Freeze weights on all layers
    for name, param in self.pretrained_model.named_parameters():
        param.requires_grad = False

    # Replace last layer (new layer has requires_grad=True by default)
    in_params = self.pretrained_model.fc.in_features
    self.pretrained_model.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(in_params, in_params)),
                                            # ('dr1_1', nn.Dropout(p=0.5)),
                                            ('bn1_1', nn.BatchNorm1d(in_params)),
                                            ('fc2', nn.Linear(in_params, 512)),
                                            ('bn2_1', nn.BatchNorm1d(512)),
                                            ('fc3', nn.Linear(512, 256)),
                                            ('bn3_1', nn.BatchNorm1d(256)),
                                            ('fc4', nn.Linear(256, num_classes))]))
    self.pretrained_model.cuda()

class GoogleNet():
  def __init__(self, num_classes=120, inputs=3):
    # Use PyTorch's pretrained AlexNet
    self.pretrained_model = models.googlenet(pretrained=True)

    # Freeze weights on all layers
    for name, param in self.pretrained_model.named_parameters():
        param.requires_grad = False

    # Replace last layer (new layer has requires_grad=True by default)
    in_params = self.pretrained_model.fc.in_features
    self.pretrained_model.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(in_params, 512)),
                                            # ('dr1_1', nn.Dropout(p=0.5)),
                                            ('bn1_1', nn.BatchNorm1d(512)),
                                            ('fc2', nn.Linear(512, 256)),
                                            ('bn2_1', nn.BatchNorm1d(256)),
                                            ('fc4', nn.Linear(256, num_classes))]))
    self.pretrained_model.cuda()

# net = AlexNet().pretrained_model
# net = ResNet18().pretrained_model
net = GoogleNet().pretrained_model

# Create training and validation dataloaders
criterion = nn.CrossEntropyLoss()
trainloader = load_dataset(dir_converted_train)
validloader = load_dataset(dir_converted_valid)

# Below function will directly train your network with the given parameters to 5 epochs
# You are also free to use function learned in task 1 to train your model here
train_loss, test_loss = start_train_test(net, trainloader, validloader, criterion)

## Making Kaggle Submission

In [0]:
##############################################################################################################
## Not So optimal Code: This can take upto 2 minutes to run: You are free to make an optimal version :)     ##
## It iterates over all test images to compute the softmax probablities from the last layer of the network  ##
##############################################################################################################
from transform import transform_testing
import PIL.Image
import torch.nn.functional as F
import numpy as np


augment_image = transform_testing()
test_data_root = dir_test + '/test_images/'
test_image_list = os.listdir(test_data_root) # list of test files 
result = []
for img_name in test_image_list:
    img = PIL.Image.open(test_data_root + img_name)
    img_tensor = augment_image(img)
    with torch.no_grad():
        output = net(img_tensor.unsqueeze_(0).cuda())
        probs = F.softmax(output, dim=1)
    result.append(probs.cpu().numpy())

all_predictions = np.concatenate(result)
print(all_predictions.shape)

In [0]:
df = pd.DataFrame(all_predictions)
file_list = os.listdir(dir_converted_train) # list of classes to be provided here
df.columns = sorted(file_list)

# insert clean ids - without folder prefix and .jpg suffix - of images as first column
test_data_root = dir_test + 'test_images/' # list of all test files here
test_image_list = os.listdir(test_data_root)
df.insert(0, "id", [e[:-4] for e in test_image_list])
df.to_csv(f"sub_1_alexnet.csv", index=False)

### TO DO!: ###
Submit the created CSV file to Kaggle, with a score(cross entropy loss) not more than __2.0__\
Take a snapshot of your rank on Kaggle Public Leaderboard and include the image here ...
For example :
![title](https://github.com/Gruppe3VDL/Gruppe3VDLws2019/blob/master/snp2.png?raw=1)

## CHALLENGE  (optional)
Compete against each other, Come up with creative ideas. Try beating the score of __0.3__. The group with minimum score gets a small prize at the time when the solutions are discussed. 


__Hints:__

1. Instead of Alexnet use pretrained resnet 18 model for better accuracy
2. Instead of a just adding the last classification layer, try adding two layers to get a better loss
3. Train some more layers at the end of the network with a very very small learning rate
4. Add Batch Normalizations or Dropout to the layers you have added, (If not present)
5. Add more augmentation to your dataset, see tranform.py file and use auto autoaugment to apply more rigorous data augmentation techniques